# 01 - Ollama Tool Loop Skeleton

This notebook is a minimal Python orchestration loop for an LLM DM using Ollama tools + JSON schema.

It mirrors the key pattern from this repo:
- build prompt context
- call model
- execute tool calls
- append tool results
- repeat until no tool calls


In [6]:
import json
import random
import re
import sys
from pathlib import Path
from typing import Any

# Ensure repo root is importable when running this notebook from its folder.
repo_root = Path.cwd()
while repo_root != repo_root.parent and not (repo_root / "orchestrator").exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from orchestrator.llm_interaction.adapter import LLMAdapter

MODEL = "llama3.1:8b"  # change to your local model
adapter = LLMAdapter(model=MODEL)


In [7]:
world_state = {
    "location": "Ravenhill",
    "time_of_day": "dusk",
    "weather": "foggy",
    "active_quests": ["Find the missing miller"],
}

def get_scene_state() -> dict[str, Any]:
    return world_state

def set_scene_state(location: str | None = None, time_of_day: str | None = None, weather: str | None = None) -> dict[str, Any]:
    if location:
        world_state["location"] = location
    if time_of_day:
        world_state["time_of_day"] = time_of_day
    if weather:
        world_state["weather"] = weather
    return world_state

def roll_dice(expression: str = "1d20") -> dict[str, Any]:
    m = re.fullmatch(r"(\d+)d(\d+)([+-]\d+)?", expression.strip())
    if not m:
        raise ValueError(f"Invalid dice expression: {expression}")
    n, sides, mod = int(m.group(1)), int(m.group(2)), int(m.group(3) or 0)
    rolls = [random.randint(1, sides) for _ in range(n)]
    return {"expression": expression, "rolls": rolls, "modifier": mod, "total": sum(rolls) + mod}

def add_quest(title: str) -> dict[str, Any]:
    world_state.setdefault("active_quests", []).append(title)
    return world_state

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_scene_state",
            "description": "Read the current DM world state.",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_scene_state",
            "description": "Update location/time/weather.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "time_of_day": {"type": "string"},
                    "weather": {"type": "string"},
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "roll_dice",
            "description": "Roll dice in NdM(+/-K) format, e.g. 1d20+3.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Dice expression"}
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_quest",
            "description": "Add a new quest objective to the active quest log.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"}
                },
                "required": ["title"],
            },
        },
    },
]

TOOL_IMPL = {
    "get_scene_state": get_scene_state,
    "set_scene_state": set_scene_state,
    "roll_dice": roll_dice,
    "add_quest": add_quest,
}


In [8]:
def execute_tool(name: str, arguments: dict[str, Any]) -> dict[str, Any]:
    if name not in TOOL_IMPL:
        return {"ok": False, "error": f"Unknown tool: {name}"}
    try:
        result = TOOL_IMPL[name](**arguments)
        return {"ok": True, "result": result}
    except Exception as e:
        return {"ok": False, "error": str(e)}


In [9]:
SYSTEM_PROMPT = (
    "You are a DnD dungeon master. Keep narrative vivid but grounded in world state. "
    "Use tools whenever world state lookup/update or dice is needed. "
    "When done, provide a concise scene update and 2-3 player choices."
)


def run_tool_loop(user_prompt: str, max_iterations: int = 10) -> dict[str, Any]:
    loop = adapter.run_tool_loop(
        stage="notebook_skeleton",
        system_prompt=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}],
        tools=TOOLS,
        tool_executor=execute_tool,
        max_iterations=max_iterations,
    )
    return {
        "status": loop["status"],
        "final_answer": loop["final_answer"],
        "rounds": loop["rounds"],
        "messages": loop["messages"],
    }


In [10]:
# Make sure Ollama is running and reachable by the `ollama` Python client.
# If your server is remote, set OLLAMA_HOST before launching the notebook.
# Example: export OLLAMA_HOST=http://your-host:11434

result = run_tool_loop("The party arrives at Ravenhill and asks if they can track the missing miller through the fog.")

print("status:", result["status"])
print("rounds:", len(result["rounds"]))
print()
print("FINAL ANSWER")
print()
print(result["final_answer"])
print()
print("TOOL CALL TRACE")
print()
for r in result["rounds"]:
    names = [c["name"] for c in r["tool_calls"]]
    print(f"iter={r['iteration']} tools={names}")


status: completed
rounds: 2

FINAL ANSWER

As the party stands at the edge of Ravenhill, the thick fog swirls around them, obscuring all but a few feet in front. The air is heavy with moisture and the sound of dripping water echoes through the mist.

The village elder approaches you, her eyes squinting against the damp air. "Ah, brave adventurers," she says, "I see you're here to help us find poor Miller Jenkins. He's been gone for three days now, and we fear something dreadful has befallen him."

She hands each of you a rough map with a crude path marked on it. "This is the route he took into the fog. Be careful, friends – Ravenhill is full of hidden dangers, especially in weather like this."

You look at the map and consider your next move.

Here are your choices:

1. **Follow Miller's trail into the fog**: Press on through the mist to see where it leads.
2. **Ask around the village for more information**: See if anyone remembers anything about Miller's disappearance or has any clues